In [68]:
#key generation
def toStr(ls):
    str = ""
    for elem in ls:
        str += elem
    return str

def p10(key):
    try:
        ls = [bit for bit in key] 
        return [ls[2], ls[4], ls[1], ls[6], ls[3], ls[9], ls[0], ls[8], ls[7], ls[6]]
    except:
        return -1
#print(toStr(p10("1010000010")))

def ls1(ls):
    try:
        return [ls[1], ls[2], ls[3], ls[4], ls[0], ls[6], ls[7], ls[8], ls[9], ls[5]]
    except:
        return -1
#print(toStr(ls1(p10("1010000010"))))
#print(toStr(ls1(ls1(ls1(p10("1010000010"))))))
    
def p8(ls):
    try:
        return [ls[5], ls[2], ls[6], ls[3], ls[7], ls[4], ls[9], ls[8]]
    except:
        return -1
k1 = p8(ls1(p10("1010000010")))
k2 = p8(ls1(ls1(ls1(p10("1010000010")))))
print("k1 =", toStr(k1))
print("k2 =", toStr(k2))

k1 = 10100100
k2 = 01000011


In [69]:
# plaintext to cyphertext

def ip(plaintext):
    try:
        ls = [bit for bit in plaintext] 
        return toStr([ls[1], ls[5], ls[2], ls[0], ls[3], ls[7], ls[4], ls[6]])
    except:
        return -1
#print( ip("12345678") ) 
    
def ipInv(plaintext):
    try:
        ls = [bit for bit in plaintext] 
        return toStr([ls[3], ls[0], ls[2], ls[4], ls[6], ls[1], ls[7], ls[5]])
    except:
        return -1
#print( ipInv( toStr( ip("12345678") ) ) )

def xor(a, b):
    if a != b:
        return "1"
    else:
        return "0"
def split(text):
    try:
        return text[:4], text[4:]
    except:
        return -1

#split("12345678")

def expansionXor(key, text):
    lis = [text[3], text[0], text[1], text[2], text[1], text[2], text[3], text[0]]
    ex = []
    for i in range(8):
        ex.append(xor(lis[i], key[i]))
    return ex
#print(toStr(expansionXor(k1, "0001")))
#print(toStr(expansionXor(k1, "1000")))
print(toStr(expansionXor(k1, "0000")))

S1 = [[1, 0, 3, 2], 
      [3, 2, 1, 0],
      [0, 2, 1, 3],
      [3, 1, 3, 2]]

S2 = [[0, 1, 2, 3], 
      [2, 0, 1, 3],
      [3, 0, 1, 0],
      [2, 1, 0, 3]]

def sbox(expanded):
    t1 = toStr(expanded)[:4]
    t2 = toStr(expanded)[4:]
    row1 = int(t1[0] + t1[3], 2)
    col1 = int(t1[1] + t1[2], 2)
    row2 = int(t2[0] + t2[3], 2)
    col2 = int(t2[1] + t2[2], 2)
    #print(row1, col1, row2, col2)
    #print(S1[row1][col1])
    #print(S2[row2][col2])
    so = "{0:02b}".format(S1[row1][col1]) + "{0:02b}".format(S2[row2][col2])
    #print(so)
    return so[1] + so[3] + so[2] + so[0]

print(sbox(expansionXor(k1, "1000")))

def f(text, key):
    l, r = split(text)
    toXor = sbox(expansionXor(key, r))
    l1 =[]
    for i in range(4):
        l1.append(xor(l[i], toXor[i]))
    return toStr(l1) + r
print(f("00001000", k1))

def sw(text):
    l, r = split(text)
    return r + l
#print(sw("00011000"))

10100100
1101
11011000


In [70]:
def simpDESencrypt(key, plaintext):
    k1 = p8(ls1(p10(key)))
    k2 = p8(ls1(ls1(ls1(p10(key)))))
    return ipInv(f(sw(f(ip(plaintext), k1)), k2))
def simpDESdecrypt(key, cyphertext):
    k1 = p8(ls1(p10(key)))
    k2 = p8(ls1(ls1(ls1(p10(key)))))
    return ipInv(f(sw(f(ip(cyphertext), k2)), k1))

plain = "10001000"
print("plaintext = ", plain)
key = "1010000010"
print("key = ", key)
cypher = simpDESencrypt(key, plain)
print("cyphertext =", cypher)
plain2 = simpDESdecrypt(key, cypher)
print("plaintext2 = ", plain2)

plaintext =  10001000
key =  1010000010
cyphertext = 00110010
plaintext2 =  10001000


In [128]:
import math
from random import shuffle
import numpy
import binascii

def isSquare(N):
    root = math.sqrt(N)
    return N == int(root + 0.5) ** 2

def findSquare(N): 
    if isSquare(N):
        return N
    else:
        nextN = math.floor(math.sqrt(N)) + 1
        return nextN * nextN
    
def columnTransposition(text, rounds):
    text2 = ''.join(c for c in text if c.isalpha())
    squared = findSquare(len(text2))
    padding = squared - len(text2)
    for i in range(padding):
        text2 += "x"
    square = math.floor(math.sqrt(squared))
    box = []
    for i in range(square):
        row =[]
        for j in range(square):
            num = i * square + j
            row.append(text2[num])
        box.append(row)
    x = [i for i in range(square)]
    shuffle(x)
    #print(x)
    for roun in range(rounds):
        newbox = []
        for i in x:
            newbox.append(box[i])
        newbox = numpy.transpose(newbox)
        box = newbox
    return box, x

matrix, random = columnTransposition("abcdefghijklmnopqrstuvx", 2)
#print(matrix)
#print(random)

def columnTranspositionDecrypt(text, rounds, order):
    n = math.floor(math.sqrt(len(text)))
    box = []
    for i in range(n):
        row =[]
        for j in range(n):
            num = i * n + j
            row.append(text[num])
        box.append(row)
    #print(x)
    newbox = []

    for roun in range(rounds):
        newbox = []
        box = numpy.transpose(box).tolist()
        counter = 0
        i = 0
        while counter <= n-1:
            if order[i] == counter:
                newbox.append(box[i])
                counter+= 1
            i += 1
            if i == n:
                i = 0
        box = newbox
    return box


#print(columnTranspositionDecrypt(matrixToString(matrix), 2, random))

def matrixToString(matrix):
    n = math.floor(math.sqrt(matrix.size))
    strin = ""
    for i in range(n):
        for j in range(n):
           strin += matrix[i][j] 
    return strin

def matrixToString2(matrix):
    n = len(matrix)
    strin = ""
    for i in range(n):
        for j in range(n):
           strin += matrix[i][j] 
    return strin

columnTransposed = matrixToString(matrix)
print(columnTransposed)

def strToBinary(string):
    newString = binascii.hexlify(string.encode('utf-8'))
    hexNum = newString.decode()
    hex2bin_map = {"0":"0000","1":"0001","2":"0010","3":"0011","4":"0100","5":"0101","6":"0110",    "7":"0111","8":"1000","9":"1001","a":"1010","b":"1011","c":"1100","d":"1101","e":"1110","f":"1111"}
    return "".join(hex2bin_map[i] for i in hexNum)

def binToStr(string):
    bin2hex_map = {"0000": "0","0001": "1","0010": "2","0011": "3","0100": "4","0101": "5","0110": "6", "0111": "7","1000": "8","1001": "9","1010": "a","1011": "b","1100": "c","1101": "d","1110": "e","1111": "f"}
    strings = [string[i: i + 4] for i in range(0, len(string), 4)]
    hexString = "".join(bin2hex_map[i] for i in strings)
    return (bytes.fromhex(hexString).decode())

print(strToBinary(columnTransposed))

moknlxxuxvceadbrtpsqhjfig
01101101011011110110101101101110011011000111100001111000011101010111100001110110011000110110010101100001011001000110001001110010011101000111000001110011011100010110100001101010011001100110100101100111


In [129]:
def enhancedDES(plaintext, key, rounds):
    matrix, random = columnTransposition(plaintext, rounds)
    binaryStr = strToBinary(matrixToString(matrix))
    strings = [binaryStr[i: i + 8] for i in range(0, len(binaryStr), 8)]
    cypher = ""
    for strr in strings:
        cypher += simpDESencrypt(key, strr)
    return cypher, random
cyphertext, random = enhancedDES("hola leonardo como estas","1010000010", 1)
print(cyphertext)
print(random)

10110111001011110010011101001100111110000010111111111000011100100010111100101111111010000010011101110010000011010101101000101111100011100111001010011101100111010100011010011101011100100000110101110111
[2, 3, 4, 0, 1]


In [130]:
def enhancedDESDecrypt(cyphertext, key, rounds, order):
    strings = [cyphertext[i: i + 8] for i in range(0, len(cyphertext), 8)]
    plain = ""
    for strr in strings:
        plain += simpDESdecrypt(key, strr)
    text = binToStr(plain)
    return matrixToString2(columnTranspositionDecrypt(text, rounds, order))

print(enhancedDESDecrypt(cyphertext,"1010000010", 1, random))

holaleonardocomoestasxxxx
